In [1]:
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/mnt/d/1ResProj/mob2crime'

In [8]:
import glob
import pandas as pd
import geopandas as gp
%matplotlib inline
import io

In [2]:
import src.mex as mex

In [3]:
folder = 'data/mexico/geography-socioeconomics/2010CensusGeography'

In [4]:
import gzip

In [5]:
import zipfile

# organize Urban Ageb and Mza

In [7]:
urban_fns = glob.glob(f'{folder}/Urban/*.zip')

In [26]:
process_list = {}

# loop over each file
for i, fn in enumerate(urban_fns):
    # open the file
    with zipfile.ZipFile(fn) as z:
        # each file could have multiple loc
        for zfn in z.namelist():
            if zfn.endswith('/'):
                continue

            zfn_split = zfn.split('/')
            length = len(zfn_split)
            loc_id_ftype_ext = zfn_split[-1]
            ftype, ext = loc_id_ftype_ext[9:].split('.')
            if ext!='shp':
                continue
            
            loc_id = loc_id_ftype_ext[:9]

            if loc_id not in process_list:
                process_list[(loc_id,ftype)] = {'loc_id': loc_id, 'fn': fn,'zfn':zfn,'ftype':ftype}


process_list = pd.DataFrame(process_list)
process_list = process_list.T.reset_index(drop=True)

In [27]:
process_list['gp_path'] = 'zip://' + process_list.fn + '/' + process_list.zfn

In [28]:
process_list['mun_id'] = process_list.loc_id.apply(lambda x: x[:5])

In [35]:
target_type = 'A'
sub_folder = 'UrbanAgeb'
os.makedirs(f'{folder}/{sub_folder}', exist_ok=True)
for mun_id, mun in process_list.groupby('mun_id'):
    to_path = f'{folder}/{sub_folder}/{mun_id}.geojson.gz'
    gpdfs  = mun[mun.ftype==target_type].gp_path.apply(lambda x: gp.read_file(x))
    gpdf = pd.concat(gpdfs.tolist(), sort=False, ignore_index=True)
    gpdf.crs = mex.crs
    with gzip.open(to_path, 'wt') as fout:
        fout.write(gpdf.to_crs(epsg=4326).to_json())


In [36]:
target_type = 'M'
sub_folder = 'UrbanMza'
os.makedirs(f'{folder}/{sub_folder}', exist_ok=True)
for mun_id, mun in process_list.groupby('mun_id'):
    to_path = f'{folder}/{sub_folder}/{mun_id}.geojson.gz'
    gpdfs  = mun[mun.ftype==target_type].gp_path.apply(lambda x: gp.read_file(x))
    gpdf = pd.concat(gpdfs.tolist(), sort=False, ignore_index=True)
    gpdf.crs = mex.crs
    with gzip.open(to_path, 'wt') as fout:
        fout.write(gpdf.to_crs(epsg=4326).to_json())


# organize Rural Ageb and Mza

In [38]:
rural_fns = glob.glob(f'{folder}/Rural/*.zip')

In [39]:
process_list = {}

# loop over each file
for i, fn in enumerate(rural_fns):
    # open the file
    with zipfile.ZipFile(fn) as z:
        # each file could have multiple loc
        for zfn in z.namelist():
            if zfn.endswith('/'):
                continue

            zfn_split = zfn.split('/')
            length = len(zfn_split)
            loc_id_ftype_ext = zfn_split[-1]
            ftype, ext = loc_id_ftype_ext[9:].split('.')
            if ext!='shp':
                continue
            
            loc_id = loc_id_ftype_ext[:9]

            if loc_id not in process_list:
                process_list[(loc_id,ftype)] = {'loc_id': loc_id, 'fn': fn,'zfn':zfn,'ftype':ftype}


process_list = pd.DataFrame(process_list)
process_list = process_list.T.reset_index(drop=True)

In [40]:
process_list['gp_path'] = 'zip://' + process_list.fn + '/' + process_list.zfn

In [41]:
process_list['mun_id'] = process_list.loc_id.apply(lambda x: x[:5])

In [44]:
import glob

In [55]:
target_type = 'L'
sub_folder = 'RuralAgeb'
os.makedirs(f'{folder}/{sub_folder}', exist_ok=True)
exist_mun_ids = set([fn.split('/')[-1][:5] for fn in glob.glob(f'{folder}/{sub_folder}/*')])

for mun_id, mun in process_list.groupby('mun_id'):
    if mun_id in exist_mun_ids: continue
    to_path = f'{folder}/{sub_folder}/{mun_id}.geojson.gz'
    gpdfs = []
    for gp_path in mun[mun.ftype==target_type].gp_path:
        try:
            gpdfs.append(gp.read_file(gp_path))
        except ValueError as e:
            print(mun_id, gp_path, e)
#         gpdfs  = mun[mun.ftype==target_type].gp_path.apply(lambda x: gp.read_file(x))
    gpdf = pd.concat(gpdfs, sort=False, ignore_index=True)
    gpdf.crs = mex.crs
    with gzip.open(to_path, 'wt') as fout:
        fout.write(gpdf.to_crs(epsg=4326).to_json())


16031 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825566197_s.zip/160310055/160310055L.shp A LinearRing must have at least 3 coordinate tuples
16061 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825566494_s.zip/160610029/160610029L.shp A LinearRing must have at least 3 coordinate tuples
16063 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825566517_s.zip/160630012/160630012L.shp A LinearRing must have at least 3 coordinate tuples


In [72]:
target_type = 'M'
sub_folder = 'RuralMza'
os.makedirs(f'{folder}/{sub_folder}', exist_ok=True)
exist_mun_ids = set([fn.split('/')[-1][:5] for fn in glob.glob(f'{folder}/{sub_folder}/*')])

for mun_id, mun in process_list.groupby('mun_id'):
    if mun_id in exist_mun_ids: continue
    to_path = f'{folder}/{sub_folder}/{mun_id}.geojson.gz'
    gpdfs = []
    for gp_path in mun[mun.ftype==target_type].gp_path:
        try:
            gpdfs.append(gp.read_file(gp_path))
        except Exception as e:
            print(mun_id, gp_path, e)
#         gpdfs  = mun[mun.ftype==target_type].gp_path.apply(lambda x: gp.read_file(x))
    gpdf = pd.concat(gpdfs, sort=False, ignore_index=True)
    gpdf.crs = mex.crs
    
    geom_null = gpdf.geometry.isnull()
    
    if geom_null.sum()!=0:
        print(mun_id, gpdf[geom_null].CVEGEO.tolist())
        gpdf = gpdf[~gpdf.geometry.isnull()]
    
    with gzip.open(to_path, 'wt') as fout:
        fout.write(gpdf.to_crs(epsg=4326).to_json())



04006 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825558345_s.zip/040060675/040060675M.shp A LinearRing must have at least 3 coordinate tuples
05011 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825558505_s.zip/050110065/050110065M.shp A LinearRing must have at least 3 coordinate tuples
10007 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825560959_s.zip/100070058/100070058M.shp A LinearRing must have at least 3 coordinate tuples
10007 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825560959_s.zip/100070067/100070067M.shp A LinearRing must have at least 3 coordinate tuples
10017 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825561055_s.zip/100170064/100170064M.shp A LinearRing must have at least 3 coordinate tuples
10018 zip://data/mexico/geography-socioeconomics/2010CensusGeography/Rural/702825561062_s.zip/100180013/100180013M.shp A LinearRing must have at 

# organize Census

In [6]:
folder_census = 'data/mexico/geography-socioeconomics/2010Census/'

loc level across the country

In [9]:
iter_nal = pd.read_csv(f'{folder_census}/National_loc_level/conjunto_de_datos/iter_00_cpv2010.csv', dtype={'entidad': str, 'mun': str, 'loc':str})

In [10]:
iter_nal['loc_id'] = iter_nal.entidad + iter_nal.mun + iter_nal['loc']

MZA level for urban area

can be downloaded by the exe file

https://www.inegi.org.mx/programas/ccpv/2010/default.html

In [11]:
fns = glob.glob(f'{folder_census}/Urban_Mza_level/*')

In [12]:
urb_mza_census = []
for fn in fns:
    with zipfile.ZipFile(fn) as zf:
        zfns = zf.namelist()
        for zfn in zfns:
            if 'conjunto_de_datos' in zfn and zfn.endswith('csv'):
                break
        csv = zf.read(zfn)
        csv = io.BytesIO(csv)    
        data = pd.read_csv(csv, dtype={'entidad': str, 'mun': str, 'loc':str, 'ageb':str, 'mza':str})
        data['loc_id'] = data.entidad + data.mun+data['loc']        
        urb_mza_census.append(data)

## all loc_id in urb_mza is in iter_nal

In [13]:

urban_loc_id = set()
for data in urb_mza_census:
    ent = data.entidad.iloc[0]
    urban_loc_id.update(data[data.mza!='000'].loc_id.tolist())
    print(set(data.loc_id) - set(iter_nal[iter_nal.entidad==ent].loc_id), len(urban_loc_id))


set() 61
set() 214
set() 358
set() 597
set() 842
set() 1307
set() 1528
set() 1619
set() 1668
set() 1748
set() 1777
set() 2386
set() 2779
set() 2861
set() 2887
set() 2967
set() 3057
set() 3161
set() 3256
set() 3319
set() 3445
set() 3506
set() 3874
set() 4001
set() 4087
set() 4105
set() 4137
set() 4195
set() 4215
set() 4403
set() 4492
set() 4525


In [14]:
iter_nal_rural = iter_nal[~iter_nal.loc_id.isin(urban_loc_id)]

## summary rows
in mza urban, rows with mza==000 are all summaries. ageb and localidad summary are urban

in loc national level, rows with loc==0000 are summaries, loc==9998and9999 are two different types of summaries

mun summaries and ent = urban + rural



In [15]:
all_match = []
for data in urb_mza_census:
    ent = data.entidad.iloc[0]
    data_no_total = data[data.mza!='000']
    
    ent_rural_no_total = iter_nal_rural[
        (~iter_nal_rural['loc'].isin(['0000', '9999','9998'])) & 
        (iter_nal_rural.entidad==ent)
    ]
    match = {}

    data_man_mza_sum = data_no_total.groupby(['entidad', 'mun', 'loc', 'ageb']).pobtot.sum()
    data_mza_sum = data[data.nom_loc=='Total AGEB urbana'].groupby(['entidad', 'mun', 'loc', 'ageb']).pobtot.sum()
    match['mza'] = (data_mza_sum == data_man_mza_sum).mean()

    data_man_ageb_sum = data_no_total.groupby(['entidad', 'mun', 'loc']).pobtot.sum()
    data_ageb_sum = data[data.nom_loc=='Total de la localidad urbana'].groupby(['entidad', 'mun', 'loc']).pobtot.sum()
    match['ageb'] = (data_ageb_sum == data_man_ageb_sum).mean()

    data_man_mun_sum = data_no_total.groupby(['entidad', 'mun']).pobtot.sum()
    data_mun_sum = data[(data.nom_loc=='Total del municipio') | (data.nom_loc=='Total del Municipio')].groupby(['entidad', 'mun']).pobtot.sum()
    match['mun'] = (data_mun_sum == data_man_mun_sum).mean()
    
    mun_sum_rural = ent_rural_no_total.groupby(['entidad', 'mun']).pobtot.sum()
    match['mun+rural'] = (data_man_mun_sum.add(mun_sum_rural,fill_value=0) == data_mun_sum).mean()


    data_man_ent_sum = data_no_total.groupby(['entidad']).pobtot.sum()
    data_ent_sum = data[(data.nom_loc=='Total de la entidad')| (data.nom_loc=='Total de la Entidad')].groupby(['entidad']).pobtot.sum()
    match['ent'] = (data_man_ent_sum == data_ent_sum).iloc[0]
    ent_sum_rural = ent_rural_no_total.groupby(['entidad']).pobtot.sum()
    match['ent+rural'] = ((data_man_ent_sum+ent_sum_rural) == data_ent_sum).mean()
    
    all_match.append(match)


In [16]:
pd.DataFrame(all_match)

,mza,ageb,mun,mun+rural,ent,ent+rural
0,1.0,1.0,0.000000,1.0,False,1.0
1,1.0,1.0,0.000000,1.0,False,1.0
2,1.0,1.0,0.000000,1.0,False,1.0
3,1.0,1.0,0.000000,1.0,False,1.0
4,1.0,1.0,0.000000,1.0,False,1.0
5,1.0,1.0,0.008000,1.0,False,1.0
6,1.0,1.0,0.000000,1.0,False,1.0
7,1.0,1.0,0.000000,1.0,False,1.0
8,1.0,1.0,0.000000,1.0,False,1.0
9,1.0,1.0,0.019608,1.0,False,1.0


## save population only

In [23]:
no_summary_rural_census = iter_nal_rural[~iter_nal_rural['loc'].isin(['0000', '9999','9998'])].drop(['longitud','latitud','altitud'],axis=1)

In [35]:
no_summary_rural_census.iloc[:, :7].to_csv(f'{folder_census}/rural_loc_pop.csv.gz', compression='gzip')

In [32]:
urb_pop = [data[data.mza!='000'].iloc[:,:9] for data in  urb_mza_census]


In [33]:
urb_pop = pd.concat(urb_pop)

In [34]:
urb_pop.sort_values(['entidad','mun','loc','ageb','mza']).reset_index(drop=True).to_csv(f'{folder_census}/urban_mza_pop.csv.gz', compression='gzip')

In [36]:
len(urb_pop)

1376969

In [40]:
urb_pop.columns[:-2]

Index(['entidad', 'nom_ent', 'mun', 'nom_mun', 'loc', 'nom_loc', 'ageb'], dtype='object')

In [44]:
urb_pop.groupby(['entidad', 'nom_ent', 'mun', 'nom_mun', 'loc', 'nom_loc', 'ageb']).pobtot.sum().to_frame().to_csv(f'{folder_census}/urban_ageb_pop.csv')